In [ ]:
# Importing packages and loading env:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
load_dotenv()

#..............................................................................................

# Importing the 3 dataframes:
df_1star = pd.read_csv('./input/one-star-michelin-restaurants.csv')
df_2star = pd.read_csv('./input/two-stars-michelin-restaurants.csv')
df_3star = pd.read_csv('./input/three-stars-michelin-restaurants.csv')

# Adding column 'stars':
df_1star['stars'] = [1]*df_1star.shape[0]
df_2star['stars'] = [2]*df_2star.shape[0]
df_3star['stars'] = [3]*df_3star.shape[0]

# Putting the three dataframes together:
df = pd.concat([df_1star, df_2star,df_3star], ignore_index=True, sort=False)

# Deleting columns that I will not need in my program: Award year & zipCode:
df = df.drop(['zipCode','year'], axis=1)

#..............................................................................................

# Types:
# print(df.dtypes) # All correct

#..............................................................................................

# Missing values:
null_cols = df.isnull().sum()
# print(null_cols[null_cols > 0])
# city: 2
# price: 176

#..............................................................................................

# City:

# Web scraping in https://guide.michelin.com to get the cities
city_list = []
for e in df['url'][df['city'].isnull() == True]:
    url = e
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    city_list.append(re.sub('\n|\s','', soup.select('.restaurant-details__heading--list')[0].text).split(',')[-2][:8])

# Filling NaN values:
index_city = 0
for i in df[df['city'].isnull() == True].index:
    df.at[i,'city'] = '{} {}'.format(city_list[index_city][:4],city_list[index_city][4:])
    index_city += 1

# df['city'].isnull().sum() # 0
    
#..............................................................................................

# Price:

# Web scraping in https://guide.michelin.com to get the prices    
prices_rest_list = []
for i in range(len(df)):
    url = df['url'][i]
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    prices = soup.select('.restaurant-details__heading-price')
    if prices:
        # prices_list.append([df['name'][i],re.sub('\n|\s','',prices[0].text)])
        prices_rest_list.append([df['name'][i], re.sub('\n|\s','',prices[0].text).split('•')[0]])

restaurants = [e[0] for e in prices_rest_list]
price = [e[1] for e in prices_rest_list]

def resub_list(array, sub_before, sub_after):
    import re
    return [re.sub(sub_before,sub_after,e) for e in array]

# Deleting thousands separator:
correct_price = resub_list(price,',','')

# Some restaurants don't have currency information. I'm deleting them.
rows_to_delete = [bool(re.match('[A-Z]{3}', correct_price[i][-3:])) for i in range(len(correct_price))]
restaurants = [restaurants[i] for i in range(len(restaurants)) if rows_to_delete[i] == True]
correct_price = [correct_price[i] for i in range(len(correct_price)) if rows_to_delete[i] == True]

# Separating price values and currency:
correct_price_2 = []
for i in range(len(correct_price)):
    correct_price_2.append('{} {}'.format(correct_price[i][:-3],correct_price[i][-3:]))
    
correct_price_3 = list(map(lambda x: x.split(' '), correct_price_2))

price = [e[0] for e in correct_price_3]
currency = [e[1] for e in correct_price_3]
price_minmax = list(map(lambda x: x.split('-'), price))
price_min = [int(e[0]) for e in price_minmax]
price_max = [int(e[1]) for e in price_minmax]

# Changing all prices to EUR:

def exchangerate_api_request(currency):
    url = "https://api.exchangerate-api.com/v4/latest/{}".format(currency)
    res = requests.get(url)
    return res

exchangerate = exchangerate_api_request('EUR').json()

currencies_to_change = list(set([e for e in currency if e != 'EUR']))
# for e in currencies_to_change:
    # if e not in list(exchangerate['rates'].keys()):
        # print(e) # MOP

# MOP TO HKD:
url = 'https://en.wikipedia.org/wiki/Macanese_pataca'
res = requests.get(url)
html = res.text
soup = BeautifulSoup(html, 'html.parser')
mop_hkd = float(soup.select('#mw-content-text > div > table:nth-child(1) > tbody > tr:nth-child(26) > td')[0].text[-4:])
# 'Hong Kong dollar (HK$) HK$1 = MOP$1.03'

price_min_eur = []
price_max_eur = []
i = 0
for e in currency:
    if e == 'MOP':
        price_min_eur.append(price_min[i]/mop_hkd/exchangerate['rates']['HKD'])  
        price_max_eur.append(price_max[i]/mop_hkd/exchangerate['rates']['HKD'])
    else:
        price_min_eur.append(price_min[i]/exchangerate['rates'][e])
        price_max_eur.append(price_max[i]/exchangerate['rates'][e])
    i += 1

# print(min(price_min_eur[0:10])) # 33.0
# print(max(price_min_eur[0:10])) # 92.0
# print(np.mean(price_min_eur[0:10])) # 64.5
# print(min(price_max_eur[0:10])) # 78.0
# print(max(price_max_eur[0:10])) # 158.0
# print(np.mean(price_max_eur[0:10])) # 116.8

df['min_price_EUR'] = [np.nan]*len(df)
df['max_price_EUR'] = [np.nan]*len(df)

for i in range(len(df)):
    for j in range(len(restaurants)):
        if df.at[i,'name'] == restaurants[j]:
            df.at[i,'min_price_EUR'] = price_min_eur[j]
            df.at[i,'max_price_EUR'] = price_max_eur[j]

# print(df['max_price_EUR'].isnull().sum()) # 26
# print(df['min_price_EUR'].isnull().sum()) # 26

# Deleting column 'price':
df = df.drop(['price'], axis=1)

# Deleting rows with missing 'price' values
df_final = df[~df['max_price_EUR'].isnull()]
df_final.reset_index(drop=True, inplace=True)

#..............................................................................................

# Missing values:
null_cols = df_final.isnull().sum()
# print(null_cols[null_cols > 0])

#..............................................................................................

# Standardizing 'cuisine' column:

for i in range(len(df_final)):
    df_final.at[i,'cuisine'] = df_final.at[i,'cuisine'].capitalize()

#..............................................................................................



In [399]:
# df_final.to_csv('./output/cleaned_enriched_df.csv')
df_final = pd.read_csv('./output/cleaned_enriched_df.csv')

In [347]:
set(df_final['region']) # La web los clasifica según estas regiones, pero no se corresponden con paises.

{'Austria',
 'California',
 'Chicago',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Finland',
 'Greece',
 'Hong Kong',
 'Hungary',
 'Ireland',
 'Macau',
 'New York City',
 'Norway',
 'Poland',
 'Rio de Janeiro',
 'Sao Paulo',
 'Singapore',
 'South Korea',
 'Sweden',
 'Taipei',
 'Thailand',
 'United Kingdom',
 'Washington DC'}

In [17]:
def battuta_request_authorized(resource):
    authToken = os.getenv("BATTUTA_API_KEY")
    if not authToken:
        raise ValueError("Missing API key")
    else:
        print("Battuta API key: ", authToken[0:4], '[...]')
    url = "http://battuta.medunes.net/api{}key={}".format(resource,authToken)
    res = requests.get(url)
    return res

In [348]:
battuta_request_authorized('/quota/?').json()

Battuta API key:  c243 [...]


{'remaining quota': 224}

In [ ]:
regions = list(set(df['region']))
regions

In [ ]:
# De las regiones, saco las que sí son paises:
data3 = []
for e in regions:
    data3.append([e,battuta_request_authorized('/country/search/?country={}&'.format(e)).json()])
data3

In [ ]:
df['country'] = ['country']*len(df)

countries = []
for i in range(len(data3)):
    if len(data3[i][1]) > 0:
        countries.append(data3[i][0])

In [84]:
for i in range(len(df)):
    for country in countries:
        if df.at[i,'region'] == country:
            df.at[i, 'country'] = country

In [ ]:
remaining_regions = [e[0] for e in data3 if e[0] not in countries]

In [ ]:
# Estas últimas no encuentro otra manera de hacerlas que manualmente...

remaining_regions_searching = [remaining_regions[0][:8], remaining_regions[1], remaining_regions[2][6:], \
                               remaining_regions[3], remaining_regions[4], 'London', remaining_regions[6], \
                               'Illinois',remaining_regions[8][:-3],remaining_regions[9][:-1]]

remaining_countries = []

for e in remaining_regions_searching:
    remaining_countries.append(battuta_request_authorized('/country/search/?city={}&'.format(e)).json()[0]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?city={}&'.format(e)).json()[0]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?country={}&'.format(e)).json()[1]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?region={}&'.format(e)).json()[1]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?city={}&'.format(e)).json()[0]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?city={}&'.format(e)).json()[0]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?city={}&'.format(e)).json()[0]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?region={}&'.format(e)).json()[0]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?city={}&'.format(e)).json()[1]['name'])
    remaining_countries.append(battuta_request_authorized('/country/search/?country={}&'.format(e)).json()[0]['name'])

remaining_countries[2] = remaining_regions[2]
remaining_countries[5] = remaining_regions[5]

In [153]:
for i in range(len(df)):
    for j in range(len(remaining_regions)):
        if df.at[i,'region'] == remaining_regions[j]:
            df.at[i, 'country'] = remaining_countries[j]

In [174]:
print(set(df['country']))

{'Ireland', 'Taiwan', 'Croatia', 'Sweden', 'Thailand', 'Greece', 'South Korea', 'United States of America', 'Macao', 'Norway', 'United Kingdom', 'Hong Kong', 'Brazil', 'Austria', 'Hungary', 'Finland', 'Singapore', 'Poland', 'Denmark', 'Czech Republic'}


In [152]:
# df.to_csv('./output/test_restaurants_prices.csv')

In [179]:
# df.to_csv('./output/test_countries.csv')